In [1]:
import numpy as np
import matplotlib.pyplot as plt
from fenics import *
from mshr import *


ModuleNotFoundError: No module named 'numpy'

In [ ]:

# Create mesh
radius = 0.1
length = 2.2
height = 0.41
cylinder = Circle(Point(0.2, 0.2), radius)
domain = Rectangle(Point(0, 0), Point(length, height)) - cylinder
mesh = generate_mesh(domain, 64)

# Define function spaces
V = VectorFunctionSpace(mesh, 'P', 2)
Q = FunctionSpace(mesh, 'P', 1)

# Define boundaries
inflow = 'near(x[0], 0)'
outflow = 'near(x[0], 2.2)'
walls = 'near(x[1], 0) || near(x[1], 0.41)'
cylinder = 'on_boundary && x[0]>0.1 && x[0]<0.3 && x[1]>0.1 && x[1]<0.3'

# Define boundary conditions
inflow_profile = ('4.0*1.5*(x[1]*(0.41 - x[1])) / pow(0.41, 2)', '0')
bcu_inflow = DirichletBC(V, Expression(inflow_profile, degree=2), inflow)
bcu_walls = DirichletBC(V, Constant((0, 0)), walls)
bcu_cylinder = DirichletBC(V, Constant((0, 0)), cylinder)
bcp_outflow = DirichletBC(Q, Constant(0), outflow)

bcu = [bcu_inflow, bcu_walls, bcu_cylinder]
bcp = [bcp_outflow]

# Define variational problem
u = TrialFunction(V)
v = TestFunction(V)
p = TrialFunction(Q)
q = TestFunction(Q)
u_n = Function(V)
u_ = Function(V)
p_n = Function(Q)
p_ = Function(Q)
f = Constant((0, 0))

mu = 1.0
rho = 1.0

# Tentative velocity step
F1 = rho * dot((u - u_n) / 0.01, v) * dx + rho * dot(dot(u_n, nabla_grad(u_n)), v) * dx \
     + mu * inner(grad(u), grad(v)) * dx - dot(f, v) * dx
a1 = lhs(F1)
L1 = rhs(F1)

# Pressure update
a2 = dot(nabla_grad(p), nabla_grad(q)) * dx
L2 = dot(nabla_grad(p_n), nabla_grad(q)) * dx - (1/0.01) * div(u_) * q * dx

# Velocity update
a3 = dot(u, v) * dx
L3 = dot(u_, v) * dx - 0.01 * dot(nabla_grad(p_ - p_n), v) * dx

# Assemble matrices
A1 = assemble(a1)
A2 = assemble(a2)
A3 = assemble(a3)

# Apply boundary conditions to matrices
for bc in bcu:
    bc.apply(A1)
for bc in bcp:
    bc.apply(A2)

# Time-stepping
t = 0
T = 2.0
while t < T:
    t += 0.01
    
    # Compute tentative velocity step
    b1 = assemble(L1)
    for bc in bcu:
        bc.apply(b1)
    solve(A1, u_.vector(), b1, 'bicgstab', 'hypre_amg')
    
    # Pressure correction
    b2 = assemble(L2)
    for bc in bcp:
        bc.apply(b2)
    solve(A2, p_.vector(), b2, 'bicgstab', 'hypre_amg')
    
    # Velocity correction
    b3 = assemble(L3)
    solve(A3, u_.vector(), b3, 'cg', 'sor')
    
    # Update previous solution
    u_n.assign(u_)
    p_n.assign(p_)

# Plot results
u_magnitude = sqrt(dot(u_n, u_n))
u_magnitude = project(u_magnitude, Q)

plt.figure()
plot(u_magnitude, title="Velocity magnitude")
plt.colorbar()
plt.show()
